In [1]:
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"

In [2]:
coord = SkyCoord(ra='20h08m06.397s', dec='+66d51m00.71s')

In [3]:
# Set the search parameters
search_radius = 25 * u.arcmin  # Replace with your desired radius
mag_limit = 15  # Replace with your desired magnitude limit

In [4]:
# Query Gaia around the coord within the radius
query = f"""
SELECT TOP 1000
    source_id, ra, dec, phot_g_mean_mag,
    phot_bp_mean_mag, phot_rp_mean_mag, 
    bp_rp, bp_g, g_rp, -- Precomputed color indices
    teff_gspphot,  -- Effective temperature
    phot_variable_flag,
    classprob_dsc_combmod_star
FROM gaiadr3.gaia_source
WHERE CONTAINS(
    POINT('ICRS', ra, dec),
    CIRCLE('ICRS', {coord.ra.deg}, {coord.dec.deg}, {search_radius.to(u.deg).value})
) = 1
AND phot_g_mean_mag < {mag_limit}
AND (classprob_dsc_combmod_star > 0.9) -- Probability it's a star
"""

# Execute the query
job = Gaia.launch_job(query)
result = job.get_results()

In [5]:
result

SOURCE_ID,ra,dec,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,bp_rp,bp_g,g_rp,teff_gspphot,phot_variable_flag,classprob_dsc_combmod_star
,deg,deg,mag,mag,mag,mag,mag,mag,K,,
int64,float64,float64,float32,float32,float32,float32,float32,float32,float32,object,float32
2249571417151115008,302.9168541102685,66.82169995859768,14.136663,14.802425,13.347171,1.4552546,0.66576195,0.7894926,5345.977,NOT_AVAILABLE,0.99998474
2249571623309571840,302.9171885072147,66.85195293540168,14.437069,14.999794,13.72113,1.2786636,0.56272507,0.71593857,5798.195,NOT_AVAILABLE,0.99995154
2249618764870607744,302.7750635525628,66.87957964539736,13.349801,14.411132,12.33817,2.0729618,1.0613308,1.011631,5040.146,NOT_AVAILABLE,0.99998933
2249618936669304704,302.93572777557165,66.87584620907832,14.800483,15.418096,14.0394335,1.3786621,0.61761284,0.7610493,5477.8276,NOT_AVAILABLE,0.99996144
2249618971029042176,302.9236683871412,66.89045752749948,12.359951,13.020082,11.570986,1.4490967,0.66013145,0.7889652,5527.666,NOT_AVAILABLE,0.9999777
2249619245904280064,302.87794283439104,66.91613464724209,14.050588,14.472099,13.246252,1.2258472,0.42151165,0.8043356,6316.972,NOT_AVAILABLE,0.9999894
...,...,...,...,...,...,...,...,...,...,...,...
2248328659774672256,301.6948657038267,67.15578323013649,14.8620825,15.689298,13.9692335,1.7200642,0.8272152,0.89284897,5038.276,NOT_AVAILABLE,0.999996


In [8]:
from astropy import units as u
from astropy.coordinates import SkyCoord
from photutils.aperture import CircularAnnulus, SkyCircularAnnulus, CircularAperture, SkyCircularAperture
from astropy.coordinates import Angle

from astropy.io import fits
from astropy.io import ascii
from astropy.wcs import WCS

from astropy.visualization import simple_norm

import matplotlib.pyplot as plt

import numpy as np

from astropy.coordinates import FK5, ICRS

from photutils.aperture import ApertureStats
from photutils.aperture import aperture_photometry

from astropy.stats import SigmaClip, mad_std

from photutils.utils import calc_total_error

import aswcs

from pathlib import Path
from tqdm import tqdm

from astropy.time import Time
from astropy.table import Table

import ccdproc as ccdp
from astropy.nddata import CCDData, StdDevUncertainty

from astropy.timeseries import LombScargle

In [27]:
*result['SOURCE_ID']

SyntaxError: can't use starred expression here (222116279.py, line 1)

In [12]:
apsize = np.array([14, 50, 90]) * 0.699 # tuple values in PIXEL (converted to arcsec)

positions = SkyCoord(result['ra'], result['dec'], frame=ICRS)
aperture = SkyCircularAperture(positions, r=apsize[0] * u.arcsec)
annuli = SkyCircularAnnulus(positions, r_in=apsize[1] * u.arcsec, r_out=apsize[2] * u.arcsec)

In [19]:
# photometry function
def sep_avg(aperture_stats : ApertureStats):
    phots = aperture_stats.sum[0] / aperture_stats.sum[1:]
    return phot, err

In [30]:
table

band,night,time,jd,airmass,2248120233601665536,2248125937318210688,2248126040397448832,2248126315275354496,2248126383994806912,2248126383994807040,2248126521433759872,2248126521433760640,2248126555793497600,2248127174268812160,2248127311707783040,2248127380427259648,2248129819968665856,2248130232285526528,2248132220852226432,2248132392650918144,2248133152863306112,2248133187223044864
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64


In [21]:
PATH = Path('/Volumes/emlaf/westep-transfer/mountpoint/space-raw/HD 191939/')

APER_SIZE_FACTOR = 1.4
BIAS = 940
BANDS = ['B', 'V', 'R', 'I']

info_table = Table(names=('band', 'night', 'time', 'jd', 'airmass'),
                   dtype=('S', 'i', 'S', 'f8', 'f8'))

In [9]:
phot_table = Table(names=result['SOURCE_ID'])

In [11]:
pixel_table = Table(names=result['SOURCE_ID'])

In [38]:
# Assuming 'result' is the Astropy table with the 'SOURCE_ID' column
# and 'table' is the existing table you want to modify.
source_ids = result['SOURCE_ID']  # List of source IDs

# For each SOURCE_ID, add an empty column to 'table'
for source_id in source_ids:
    # Convert source_id to string for column naming
    col_name = str(source_id)
    
    # Add an empty column (filled with None or np.nan for numerical data)
    # Assuming you want the new column to match the length of 'table'
    table[col_name] = [None] * len(table)  # or np.nan for numerical placeholders

# Now 'table' has empty columns named after each SOURCE_ID
print(table)

band night time  jd airmass ... 2248130232285526528 2248132220852226432 2248132392650918144 2248133152863306112 2248133187223044864
---- ----- ---- --- ------- ... ------------------- ------------------- ------------------- ------------------- -------------------


In [22]:
for source_id in result['SOURCE_ID']:
    info_table[str(source_id)] = [np.nan] * len(info_table)  # or np.nan for numerical placeholders

In [23]:
info_table

band,night,time,jd,airmass,2249571417151115008,2249571623309571840,2249618764870607744,2249618936669304704,2249618971029042176,2249619245904280064,2249620517217262976,2249620689015959808,2249621204412177536,2249621410570604544,2249621410570604800,2249621612432435584,2249621822887754240,2249622544441958912,2249623231636590464,2249623747032785536,2249623850111997440,2249624739168184320,2249625190141803136,2249625739897620864,2249625774257357824,2249625877336564992,2249626319716146944,2249626358372886656,2249626427092354944,2249626667610545664,2249626839409231232,2249626908128700928,2249626942488443904,2249626049135258752,2249636013459348864,2249636150898307456,2249636387119490688,2249636425776205696,2249636803733329536,2249636803733329664,2249636769373596928,2249636872453545728,2249636902515564160,2249637216050180992,2249637422208600576,2249637731446254848,2249637765805997568,2249638075043636608,2249638178122875648,2249638212482612224,2249639277634486912,2249639758670815488,2249640716446521216,2249640892542165248,2249640995621386624,2249641236139552128,2249641236139552384,2249641373578506880,2249641373578508032,2249641785895355520,2249642576169357824,2248058660950328960,2248059038907452672,2248059245065916160,2248059245065916416,2248060482016498688,2248060791254176768,2248060963052870528,2248061100491864064,2248061341010031872,2248066731191828224,2248067388324010624,2248067800640871168,2248067972439591552,2248068380459319680,2248068419116191616,2248068517898292096,2248068522195407872,2248068590914884224,2248068900152551936,2248069140670720640,2248069759146010240,2248069930944701952,2248069965304460800,2248082472248963712,2248082918925563136,2248084980509819136,2248085255387769088,2248085805143585152,2248085908222850816,2248086389259188608,2248086457978698624,2248086870295523456,2248086973374738688,2248087110814482688,2248087145173429120,2248086045661803392,2248095150992412928,2248096284863828608,2248096559741734912,2248097178216980224,2248097246936458496,2248097384375461760,2248098002848533120,2248098415167610112,2248098552606598400,2248098552606598528,2248098690045516672,2248098861844212352,2248099102362414464,2248099102362414848,2248099102362415104,2248099239801367168,2248099480319537280,2248099514679275776,2248099617758525440,2248101095227201920,2248101301385632000,2248101507544061696,2248101748062265344,2248101748062265600,2248100133154564224,2248102022940171264,2248102091659646720,2248102435256996096,2248103019372617728,2248104187603761920,2248106008669834496,2248106489706169984,2248106936382766848,2248107279980186496,2248108070254169984,2248108516930809088,2248108585650285312,2248108688729500032,2248108997967143936,2248109238485311744,2248109272845050368,2248109341564528768,2248109856960527872,2248110509795591552,2248110612874845184,2248110750313759616,2248111059551403136,2248111231350094464,2248111678026733440,2248111884185162368,2248112193422810880,2248112227782576640,2248112257845204736,2248112326564671104,2248112365221501568,2248114289366881920,2248115938634349824,2248116007353849600,2248116110433064960,2248117313023929856,2248117381743406208,2248117519182392192,2248117897139476864,2248118206377158784,2248119168449777408,2248119198511385984,2248119271528993152,2248119374608209152,2248119512047161344,2248119615126393856,2248119683845870080,2248119821284805888,2248120233601665536,2248120366743588096,2248121741133168896,2248122290888979456,2248122466984708992,2248122673143143168,2248122844941831680,2248123360337775616,2248123532136467200,2248123738294925824,2248123837077077632,2248123841374140672,2248123875733851392,2248123910093616896,2248124077594234624,2248124184971495936,2248124562928617600,2248125043964981504,2248125937318210688,2248126040397448832,2248126315275354496,2248126383994806912,2248126383994807040,2248126418354543360,2248126521433759872,2248126521433760640,2248126555793497600,2248127174268812160,2248127311707783040,2248127380427259648,2248127724024572416,2248128170701198336,2248128411219366144,2248128926615

In [49]:
# multi band - exec cell

# list of dates
date_list = list(PATH.glob('[!.]*'))

# Initialize the progress bar
progress_bar = tqdm(total=len(date_list))

for date_path in date_list:
    # date string
    date = date_path.name

    # # open calibration files
    # try:
    #     b = fits.open(BIAS_PATH / date / 'master_bias.fit', output_verify='warn')
    #     ccd_bias_data = CCDData(b[0].data,
    #                             StdDevUncertainty(b[2].data, unit='adu'),
    #                             b[1].data,
    #                             unit='adu')
    # except (ValueError, FileNotFoundError):
    #     progress_bar.update(1)
    #     continue

    for file in date_path.glob("[!._]*.wcs"):
        # read WCS
        try:
            wcs = aswcs.ini_to_wcs(file.with_suffix('.ini'))
        
            with fits.open(file.with_suffix('.fts'), output_verify='warn') as f:
                # # get data and bias subtraction
                # data = ccdp.subtract_bias(CCDData(f[0].data, unit='adu'), ccd_bias_data).data
                data = CCDData(f[0].data, unit='adu', wcs=wcs)
                
                # centroiding and setting aperture size
                raw_aperstats = ApertureStats(data, aperture)
                centroids = raw_aperstats.centroid
                aper_size = raw_aperstats.fwhm[0].value * APER_SIZE_FACTOR
            
                # get background
                sigclip = SigmaClip(sigma=3.0, maxiters=10)
                bkg_stats = ApertureStats(data,
                                          CircularAnnulus(centroids, r_in=aper_size * 1.7, r_out=aper_size * 1.7 * 3 / 2),
                                          sigma_clip=sigclip)
        
                # photometry
                aper_stats_bkgsub = ApertureStats(data.data,
                                                  CircularAperture(centroids, r=aper_size),
                                                  local_bkg=bkg_stats.median.value,
                                                  error=calc_total_error(f[0].data.astype(np.float64), 0, 1.85))
                # reject saturation
                if np.max(aper_stats_bkgsub.max) > 64000:
                    print("Saturation")
                    # raise ValueError('Saturation')
                
                phot_result_list = []
                pixel_pos_list = []
                for i in aper_stats_bkgsub:
                    phot_result_list.append(i.sum)
                    pixel_pos_list.append(i.centroid)
                
                # save to table
                info_table.add_row([f[0].header['FILTER'],
                                    date,
                                    f[0].header['DATE-OBS'],
                                    f[0].header['JD'],
                                    f[0].header['AIRMASS']])
                phot_table.add_row(phot_result_list)
                pixel_table.add_row(pixel_pos_list)
                    
                
                # supercomp_flux = np.sum(aper_stats_bkgsub.sum[1:])
                # target_var = aper_stats_bkgsub.sum_err[0]
                # supercomp_var = np.sum(aper_stats_bkgsub.sum_err[1:] ** 2)
                    
                
        except Exception as e:
            raise e
            
    progress_bar.update(1)

progress_bar.close()

  8%|████████                                                                                                 | 11/143 [00:58<10:24,  4.73s/it]

Saturation
Saturation
Saturation
Saturation
Saturation
Saturation
Saturation
Saturation
Saturation
Saturation


 22%|██████████████████████▊                                                                                  | 31/143 [02:39<07:43,  4.13s/it]

Saturation
Saturation
Saturation
Saturation
Saturation
Saturation
Saturation
Saturation
Saturation
Saturation


 22%|███████████████████████▍                                                                                 | 32/143 [02:41<06:42,  3.63s/it]

...

 80%|██████████████████████████████████████████████████████████████████████████████████▉                     | 114/143 [09:14<03:13,  6.67s/it]

Saturation
Saturation
Saturation
Saturation


 87%|██████████████████████████████████████████████████████████████████████████████████████████▉             | 125/143 [10:48<02:55,  9.76s/it]

Saturation
Saturation
Saturation
Saturation
Saturation
Saturation
Saturation


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 143/143 [12:27<00:00,  5.23s/it]


In [47]:
table_save = table.copy()

In [50]:
table.write('full_table_hd19.fits')

In [52]:
import pickle
with open('full_table.pkl', 'wb') as f:
    pickle.dump(table, f, pickle.HIGHEST_PROTOCOL)

In [51]:
table

band,night,time,jd,airmass,2248120233601665536,2248125937318210688,2248126040397448832,2248126315275354496,2248126383994806912,2248126383994807040,2248126521433759872,2248126521433760640,2248126555793497600,2248127174268812160,2248127311707783040,2248127380427259648,2248129819968665856,2248130232285526528,2248132220852226432,2248132392650918144,2248133152863306112,2248133187223044864
bytes32,int32,bytes32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
B,20230627,2023-06-28T02:14:36.070,2460123.5934730326,1.36117194472,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
B,20230627,2023-06-28T02:14:36.070,2460123.5934730326,1.36117194472,3634.4699491188844,5401.177856409149,809.6454394804232,781242.511494014,1668.1276916912893,4160.419564695545,5817.510337121565,1277.0368576411715,4548.740352295337,3973.838388417552,17952.74955574747,1702.9463337766874,4518.469278034454,26623.557283888527,19515.530797230043,10981.035306395344,14566.049221392284,1191.5494585989322
V,20230627,2023-06-28T02:06:02.290,2460123.5875265044,1.38040740207,4414.036495327581,3817.6904626613164,2953.9988491535114,670128.8665214505,2510.864350093072,3124.1847272954183,5121.952016547708,3197.4845764478077,4325.739891884306,4089.100870897956,18666.64909567086,3050.5394729874074,6671.723300423163,22027.86707605754,19703.94162216918,16275.44387825331,10673.65873431477,1372.562134948274
R,20230627,2023-06-28T02:24:59.200,2460123.6006851853,1.33984954121,-1434.4976182645269,2285.4673468351884,1700.8559099039019,272298.2906990053,700.9729581249867,1450.9889230249887,2345.5378583000083,1016.1009120198637,840.4024627354823,2274.823566108802,8613.727959206191,-429.14206441211275,5968.132469992431,7320.843354905266,8304.41771732481,7513.291131574446,2139.5028097878585,-62.6761484780111
I,20230627,2023-06-28T02:21:57.100,2460123.5985775464,1.34608516429,513.1059196982337,917.6488484111348,1357.4057273860371,101355.68933261678,39.530540002963676,480.12002307102955,1860.506082290305,-12.337603680064433,388.59780907881736,30.40352123204201,4084.724951806321,645.2143770442583,1879.1767546230435,3064.1319798820978,3049.486326353584,5606.164819576654,1804.5297102650975,168.22990331501308
V,20230627,2023-06-28T02:04:31.410,2460123.586474653,1.38380751076,2675.2311923196385,3254.2649521776602,4384.797482791831,666611.1923334205,3282.2689346073494,3339.0416978849635,3998.684163567963,1524.5905274905867,3089.653256624877,3503.7205401105334,17878.256335454535,2113.72970791655,5985.661009165836,21670.14315044051,18095.823533695595,15206.196823371187,8702.379535022683,2646.253011122413
I,20230627,2023-06-28T02:21:24.699,2460123.598202546,1.34720558865,1502.8822472147447,1883.4522314886397,1106.298817818224,131422.20051717057,2214.084151316534,702.1747723973524,114.77563887755122,162.57722664507878,595.9655404249518,2331.168064406839,4454.238853516097,464.04208666473517,3618.329774155321,4348.889552102157,4994.972932781016,5759.177078846949,1038.901563634383,2218.4595098922878
R,20230627,2023-06-28T02:24:27.010,2460123.6003126157,1.34094133399,3108.1953776706214,4873.568105202756,3569.0773447594647,560912.8269567775,2039.509317006953,3427.8740188846873,3076.4336346920363,2286.3432862049954,2688.7044795399906,4709.925186325291,18069.636087945255,1162.5123946314347,8112.080284969762,16915.375179106275,14751.365929745223,16211.733473742937,7292.650304342942,1807.8459833678467
V,20230627,2023-06-28T02:05:25.789,2460123.587104051,1.38174939343,3448.2040963652057,4057.3520567469704,4355.086625049777,670370.335069656,934.5383021254966,3644.874553318567,4613.926560581938,2137.9617168348236,3336.7197199383786,5079.301815161589,19168.283061077957,2012.8201255527715,7374.868817427848,23711.60665166724,18189.82005227616,15587.990723794966,9222.404560547624,2309.557610401054
